In [23]:
import os
import sys
sys.path.append(os.path.abspath("../src"))


In [20]:
%pwd

'c:\\Users\\Rayaan\\Desktop\\ML_Project\\End-to-End-ML-Project-with-ML-Flow\\research'

In [9]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [26]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [28]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH,
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=Path(config.root_dir),
            source_URL=config.source_URL,
            local_data_file=Path(config.local_data_file),
            unzip_dir=Path(config.unzip_dir),
        )

        return data_ingestion_config
    
   

In [29]:
import os
import urllib.request as request
import zipfile
from mlProject import logger
from mlProject.utils.common import get_size

In [32]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file,
            )
            logger.info(f"{filename} downloaded! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

        return self.config.local_data_file

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
                zip_ref.extractall(unzip_path)
            

In [35]:
import os

# Go to the project root
os.chdir(r"C:\Users\Rayaan\Desktop\ML_Project\End-to-End-ML-Project-with-ML-Flow")

# Verify
print(os.getcwd())  # should print the project root path


C:\Users\Rayaan\Desktop\ML_Project\End-to-End-ML-Project-with-ML-Flow


In [43]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-08-27 13:27:13,608: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-08-27 13:27:13,613: INFO: common: yaml file: params.yaml loaded successfully]
[2025-08-27 13:27:13,616: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-08-27 13:27:13,618: INFO: common: created directory at: artifacts]
[2025-08-27 13:27:13,621: INFO: common: created directory at: artifacts/data_ingestion]
[2025-08-27 13:27:14,496: INFO: 3297616639: artifacts\data_ingestion\data.zip downloaded! with following info: 
Connection: close
Content-Length: 23531
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "573a2f3952efd60cd556c1df37b2f46738e66caec964f046751ab96b314fd33f"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: D605:3EA1D2:74341:14F785:68AEB48A
Accept-Ranges: bytes
Date: Wed, 2

In [45]:
import zipfile
import os

# Paths
outer_zip = "artifacts/data_ingestion/data.zip"
unzip_dir = "artifacts/data_ingestion"

# Ensure directory exists
os.makedirs(unzip_dir, exist_ok=True)

# Extract outer zip
with zipfile.ZipFile(outer_zip, 'r') as zip_ref:
    zip_ref.extractall(unzip_dir)

# Check files after extraction
print("After first extraction:", os.listdir(unzip_dir))

# Find inner zip
inner_zip = os.path.join(unzip_dir, "winequality-data.zip")

# Extract inner zip
with zipfile.ZipFile(inner_zip, 'r') as zip_ref:
    zip_ref.extractall(unzip_dir)

print("After second extraction:", os.listdir(unzip_dir))


After first extraction: ['data.zip', 'winequality-data.zip']
After second extraction: ['data.zip', 'winequality-data.zip', 'winequality-red.csv']
